In [1]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPEN_AI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

## Tools

In [7]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [6]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [8]:
@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

@tool
def calculate_uppercase_tool(a: str) -> int:
    """The function calculates the number of uppercase characters in the input string."""
    return sum(1 for c in a if c.isupper())

## Memory

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

### Long-term memory

In [3]:
loader = WebBaseLoader("https://neurons-lab.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "neurons_lab_search",
    "Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!",
)

### Short-term memory

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [10]:
message_history = ChatMessageHistory()

### Sensory memory

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [24]:
template ="""
Step1 :
I have a problem related to {agent_scratchpad}. Could you brainstorm three distinct solutions?
Please consider a variety of factors such as cost, time, and resources.
A:
"""

prompt = PromptTemplate(
    input_variables=["agent_scratchpad"],
    template = template                      
)

## Agent

In [36]:
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI
from langchain import hub

from langchain.agents import create_openai_functions_agent
from langchain.chat_models import ChatOpenAI

In [37]:
tools = [retriever_tool, tavily_tool, calculate_length_tool, calculate_uppercase_tool]
for t in tools:
    print(t.name, t.description)

neurons_lab_search Search for information about Neurons Lab. For any questions about Neurons Lab, you must use this tool!
tavily_search_results_json A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
calculate_length_tool calculate_length_tool(a: str) -> int - The function calculates the length of the input string.
calculate_uppercase_tool calculate_uppercase_tool(a: str) -> int - The function calculates the number of uppercase characters in the input string.


In [39]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(
        llm,
        tools,
        prompt
    )
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="agent_scratchpad",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"agent_scratchpad": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Streaming LangChain activity to W&B at https://wandb.ai/rachnogstyle/nlw_jan24/runs/gucv1rau
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.




> Entering new AgentExecutor chain...


wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


Sure, I can help with that. To provide you with three distinct solutions related to the problem you have mentioned, I need to know more details about the specific problem you are facing. Could you please provide more information or context about the issue related to [] so that I can brainstorm relevant solutions for you?

> Finished chain.


{'agent_scratchpad': "hi! I'm bob",
 'chat_history': [],
 'output': 'Sure, I can help with that. To provide you with three distinct solutions related to the problem you have mentioned, I need to know more details about the specific problem you are facing. Could you please provide more information or context about the issue related to [] so that I can brainstorm relevant solutions for you?'}

In [28]:
agent_with_chat_history.invoke(
    {"agent_scratchpad": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


Sure, I can help with that. Let's brainstorm three distinct solutions for the problem related to []. 

Solution 1: Implementing a Software Solution
One solution could be to develop a software solution that addresses the problem. This could involve creating a custom software application or using existing software tools to automate processes, improve efficiency, and streamline operations. While this solution may require an initial investment in development and implementation, it could potentially save time and resources in the long run by increasing productivity and reducing manual work.

Solution 2: Training and Education
Another solution could involve providing training and education to employees or stakeholders to address the problem. By investing in training programs, workshops, or seminars, individuals can gain the knowledge and skills needed to effectively deal with the issue at hand. While this solution may require time and resources to organize and execute, it can lead to long-te

{'agent_scratchpad': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Sure, I can help with that. To provide you with three distinct solutions related to the problem you have mentioned, I need to know more details about the specific problem you are facing. Could you please provide more information or context about the issue related to [] so that I can brainstorm relevant solutions for you?')],
 'output': "Sure, I can help with that. Let's brainstorm three distinct solutions for the problem related to []. \n\nSolution 1: Implementing a Software Solution\nOne solution could be to develop a software solution that addresses the problem. This could involve creating a custom software application or using existing software tools to automate processes, improve efficiency, and streamline operations. While this solution may require an initial investment in development and implementation, it could potentially save time and resources in the long run by